<a href="https://colab.research.google.com/github/jcjcchang/2nd-ML100Days/blob/master/homework/Day_027_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [17]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')


from google.colab import drive
drive.mount('gdrive')

df = pd.read_csv('gdrive/My Drive/draft-ds/titanic_train.csv')


train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
#Pclass , Fare
"""
Your Code Here
"""
#df_temp = (df.isna()==True)
#df_temp.describe()
#df['Pclass'] = df['Pclass'].fillna('None')
#df['Fare'] = df['Fare'].fillna('None')
#不用 fillna 因為 Pclass & Fare 都沒有 NaN 

df_mean = df.groupby(['Pclass'])['Fare'].mean().reset_index()
df_mode = df.groupby(['Pclass'])['Fare'].apply(lambda x : x.mode()[0]).reset_index()
df_median = df.groupby(['Pclass'])['Fare'].median().reset_index()
df_max = df.groupby(['Pclass'])['Fare'].max().reset_index()
df_temp = pd.merge(df_mean,df_mode,how='left',on=['Pclass'])
df_temp = pd.merge(df_temp,df_median,how='left',on=['Pclass'])
df_temp = pd.merge(df_temp,df_max,how='left',on=['Pclass'])
df_temp.columns = ['Pclass','Fare_mean','Fare_mode','Fare_median','Fare_max']
df_temp



,Pclass,Fare_mean,Fare_mode,Fare_median,Fare_max
0,1,84.154687,26.55,60.2875,512.3292
1,2,20.662183,13.00,14.2500,73.5000
2,3,13.675550,8.05,8.0500,69.5500


In [19]:
df = pd.merge(df,df_temp,how='left',on='Pclass')
df = df.drop(['Fare'],axis = 1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Cabin,Embarked,Fare_mean,Fare_mode,Fare_median,Fare_max
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,NaN,S,13.675550,8.05,8.0500,69.5500
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,C85,C,84.154687,26.55,60.2875,512.3292
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,NaN,S,13.675550,8.05,8.0500,69.5500
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,C123,S,84.154687,26.55,60.2875,512.3292
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,NaN,S,13.675550,8.05,8.0500,69.5500


In [20]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

8 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max']



,Pclass,Age,SibSp,Parch,Fare_mean,Fare_mode,Fare_median,Fare_max
0,3,22.0,1,0,13.675550,8.05,8.0500,69.5500
1,1,38.0,1,0,84.154687,26.55,60.2875,512.3292
2,3,26.0,0,0,13.675550,8.05,8.0500,69.5500
3,1,35.0,1,0,84.154687,26.55,60.2875,512.3292
4,3,35.0,0,0,13.675550,8.05,8.0500,69.5500


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [22]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
# 沒有這四個新特徵的 dataframe 稱為 df_minus
df_minus = df[['Pclass','Age','SibSp','Parch']]
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6959983856936712

In [23]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6971283291964961